In [10]:
using IPGBs
using JuMP
using LinearAlgebra
using Random

In [6]:
Random.seed!(0)
n = 5
c = rand(1:10, 1, n)
A = rand(1:10, 1, n)
b = [round(Int, sum(A) / 2)]

1-element Vector{AffExpr}:
 5 x[1] + x[2] + 9 x[3] + x[4] + 7 x[5]

In [9]:
gr = IPGBs.FourTi2.graver(A, [true, true, true, true, true]);

In [11]:
function lawrence_lifting(A)
    m, n = size(A)
    I5 = I(5)
    Z5 = zeros(Int, m, n) 
    return [A Z5; I5 I5]
end

lawrence_lifting (generic function with 1 method)

In [44]:
gr_lift = IPGBs.FourTi2.graver(lawrence_lifting(A), [true for _ in 1:10])
gr_lift

51×10 Matrix{Int64}:
 0   1   0   0  -2   0  -1   0   0   2
 0   0   1   0  -8   0   0  -1   0   8
 0   0   0   1  -3   0   0   0  -1   3
 1   0   0   0  -2  -1   0   0   0   2
 1  -1   0   0   0  -1   1   0   0   0
 1   0   0  -1   1  -1   0   0   1  -1
 1   0  -1   0   6  -1   0   1   0  -6
 0   1   0  -1   1   0  -1   0   1  -1
 0   1  -1   0   6   0  -1   1   0  -6
 0   0   1  -1  -5   0   0  -1   1   5
 1   1   0  -1  -1  -1  -1   0   1   1
 1   0  -1   1   3  -1   0   1  -1  -3
 1   1  -1   0   4  -1  -1   1   0  -4
 ⋮                   ⋮              
 0   4  -1   0   0   0  -4   1   0   0
 1   2   0  -2   0  -1  -2   0   2   0
 2   1   0  -2   0  -2  -1   0   2   0
 0   3   0  -2   0   0  -3   0   2   0
 3   0   0  -2   0  -3   0   0   2   0
 1   1   1  -4   0  -1  -1  -1   4   0
 0   2   1  -4   0   0  -2  -1   4   0
 2   0   1  -4   0  -2   0  -1   4   0
 0   0   2  -5  -1   0   0  -2   5   1
 0   1   2  -6   0   0  -1  -2   6   0
 1   0   2  -6   0  -1   0  -2   6   0
 0   0

In [40]:
for i in 1:size(gr, 1)
    println(gr[i, :])
end

[1, 0, 0, 0, -2]
[0, 1, 0, 0, -2]
[0, 0, 1, 0, -8]
[0, 0, 0, 1, -3]
[1, -1, 0, 0, 0]
[1, 0, 0, -1, 1]
[1, 0, -1, 0, 6]
[0, 1, 0, -1, 1]
[0, 1, -1, 0, 6]
[0, 0, 1, -1, -5]
[1, 1, 0, -1, -1]
[1, 0, -1, 1, 3]
[1, 1, -1, 0, 4]
[2, 0, 0, -1, -1]
[2, 0, -1, 0, 4]
[0, 2, 0, -1, -1]
[0, 1, -1, 1, 3]
[0, 2, -1, 0, 4]
[0, 0, 1, -2, -2]
[1, 0, -1, 2, 0]
[2, 0, -1, 1, 1]
[1, 1, -1, 1, 1]
[2, 1, -1, 0, 2]
[3, 0, -1, 0, 2]
[1, 2, -1, 0, 2]
[0, 1, -1, 2, 0]
[0, 2, -1, 1, 1]
[0, 3, -1, 0, 2]
[0, 0, 1, -3, 1]
[1, 0, 1, -3, -1]
[1, 2, -1, 1, -1]
[1, 3, -1, 0, 0]
[2, 1, -1, 1, -1]
[2, 2, -1, 0, 0]
[3, 0, -1, 1, -1]
[3, 1, -1, 0, 0]
[4, 0, -1, 0, 0]
[0, 1, 1, -3, -1]
[0, 3, -1, 1, -1]
[0, 4, -1, 0, 0]
[1, 2, 0, -2, 0]
[2, 1, 0, -2, 0]
[0, 3, 0, -2, 0]
[3, 0, 0, -2, 0]
[1, 1, 1, -4, 0]
[0, 2, 1, -4, 0]
[2, 0, 1, -4, 0]
[0, 0, 2, -5, -1]
[0, 1, 2, -6, 0]
[1, 0, 2, -6, 0]
[0, 0, 3, -8, 0]


In [51]:
new_A = lawrence_lifting(A)
new_b = [10000, 10000, 10000, 10000, 10000, 10000]
new_c = [1 1 1 1 1 1 1 1 1 1]
G = groebner_basis(new_A, new_b, new_c; truncation_type=:None)

40-element Vector{Vector{Int64}}:
 [-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]
 [-1, 0, 0, 0, 1, 1, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0]
 [-1, 0, 0, 0, 2, 1, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0]
 [-1, 0, 0, 1, 0, 1, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0]
 [-1, 0, 1, 0, 0, 1, 0, -1, 0, 0, -6, 0, 0, 0, 0, 0]
 [-1, 1, 0, 0, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0]
 [0, -1, 0, 0, 1, 0, 1, 0, 0, -1, 1, 0, 0, 0, 0, 0]
 [0, -1, 0, 0, 2, 0, 1, 0, 0, -2, 0, 0, 0, 0, 0, 0]
 [0, -1, 0, 1, 0, 0, 1, 0, -1, 0, -1, 0, 0, 0, 0, 0]
 [0, -1, 1, 0, 0, 0, 1, -1, 0, 0, -6, 0, 0, 0, 0, 0]
 [0, 0, -1, 0, 0, 0, 0, 1, 0, 0, 8, 0, 0, 0, 0, 0]
 [0, 0, -1, 0, 1, 0, 0, 1, 0, -1, 7, 0, 0, 0, 0, 0]
 ⋮
 [0, 0, -1, 2, 2, 0, 0, 1, -2, -2, 0, 0, 0, 0, 0, 0]
 [0, 0, -1, 3, 0, 0, 0, 1, -3, 0, -1, 0, 0, 0, 0, 0]
 [0, 0, 0, -1, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0]
 [0, 0, 0, -1, 1, 0, 0, 0, 1, -1, 2, 0, 0, 0, 0, 0]
 [0, 0, 0, -1, 2, 0, 0, 0, 1, -2, 1, 0, 0, 0, 0, 0]
 [0, 0, 0, -1, 3, 0, 0, 0

In [49]:
IPGBs.FourTi2.groebner(new_A, new_c)

51×10 Matrix{Int64}:
 -4   0   1   0   0  4  0  -1   0   0
 -3  -1   1   0   0  3  1  -1   0   0
 -3   0   0   2   0  3  0   0  -2   0
 -3   0   1  -1   1  3  0  -1   1  -1
 -3   0   1   0  -2  3  0  -1   0   2
 -2  -2   1   0   0  2  2  -1   0   0
 -2  -1   0   2   0  2  1   0  -2   0
 -2  -1   1  -1   1  2  1  -1   1  -1
 -2  -1   1   0  -2  2  1  -1   0   2
 -2   0  -1   4   0  2  0   1  -4   0
 -2   0   0   1   1  2  0   0  -1  -1
 -2   0   1  -1  -1  2  0  -1   1   1
 -2   0   1   0  -4  2  0  -1   0   4
  ⋮                  ⋮             
  0  -1   0   0   2  0  1   0   0  -2
  0  -1   0   1  -1  0  1   0  -1   1
  0  -1   1  -2   0  0  1  -1   2   0
  0  -1   1  -1  -3  0  1  -1   1   3
  0  -1   1   0  -6  0  1  -1   0   6
  0   0  -3   8   0  0  0   3  -8   0
  0   0  -2   5   1  0  0   2  -5  -1
  0   0  -1   0   8  0  0   1   0  -8
  0   0  -1   1   5  0  0   1  -1  -5
  0   0  -1   2   2  0  0   1  -2  -2
  0   0  -1   3  -1  0  0   1  -3   1
  0   0   0  -1   3  0  0   0  

In [45]:
for g in G
    println(g)
end

[-2, 0, 1, 0, 0, 2, 0, -1, 0, 0, -4, 0, 0, 0, 0, 0]
[-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]
[-1, 0, 0, 1, 0, 1, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0]
[-1, 0, 1, 0, -1, 1, 0, -1, 0, 1, -5, 0, 0, 0, 0, 0]
[-1, 1, 0, 0, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0]
[0, -1, 0, 1, 0, 0, 1, 0, -1, 0, -1, 0, 0, 0, 0, 0]
[0, 0, -1, 0, 0, 0, 0, 1, 0, 0, 8, 0, 0, 0, 0, 0]
[0, 0, -1, 0, 1, 0, 0, 1, 0, -1, 7, 0, 0, 0, 0, 0]
[0, 0, -1, 1, 0, 0, 0, 1, -1, 0, 5, 0, 0, 0, 0, 0]
[0, 0, -1, 1, 1, 0, 0, 1, -1, -1, 4, 0, 0, 0, 0, 0]
[0, 0, -1, 2, 0, 0, 0, 1, -2, 0, 2, 0, 0, 0, 0, 0]
[0, 0, -1, 2, 1, 0, 0, 1, -2, -1, 1, 0, 0, 0, 0, 0]
[0, 0, -1, 3, 0, 0, 0, 1, -3, 0, -1, 0, 0, 0, 0, 0]
[0, 0, 0, -1, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0]
[0, 0, 0, 0, -1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, -1, 0,